In [1]:
from tokenizers import BertWordPieceTokenizer
import pandas as pd

In [4]:
df = pd.read_pickle('../preprocessed/tweets_bert.pkl')
df.head()

,tweet,label,tokens
0,as a woman you shouldnt complain about cleanin...,0,"[101, 2004, 1037, 2450, 2017, 5807, 2102, 1761..."
1,boy dats coldtyga dwn bad for cuffin dat hoe i...,0,"[101, 2879, 23755, 2015, 3147, 3723, 3654, 104..."
2,dawg you ever fuck a bitch and she sta to cry ...,0,"[101, 4830, 27767, 2017, 2412, 6616, 1037, 774..."
3,she look like a tranny,0,"[101, 2016, 2298, 2066, 1037, 25283, 4890, 102]"
4,the shit you hear about me might be true or it...,0,"[101, 1996, 4485, 2017, 2963, 2055, 2033, 2453..."


## prepare the data

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert token sequences to tensors and pad them
train_input_ids = [torch.tensor(tokens) for tokens in train_df['tokens']]
train_input_ids = pad_sequence(train_input_ids, batch_first=True)
train_labels = torch.tensor(train_df['label'].tolist())

val_input_ids = [torch.tensor(tokens) for tokens in val_df['tokens']]
val_input_ids = pad_sequence(val_input_ids, batch_first=True)
val_labels = torch.tensor(val_df['label'].tolist())

# Create TensorDatasets
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_labels)
val_dataset = torch.utils.data.TensorDataset(val_input_ids, val_labels)

# Define batch size
batch_size = 32

# Create DataLoader for training and validation sets
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


### Define BERT model

In [15]:
from transformers import BertForSequenceClassification

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Optionally, freeze BERT layers if you want to fine-tune only the classification head
# for param in model.bert.parameters():
#     param.requires_grad = False


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### training loop

In [16]:
from transformers import AdamW
from tqdm import tqdm

# Define optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Define number of epochs
num_epochs = 3

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}'):
        input_ids, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_dataloader)
    print(f'Average loss: {avg_loss}')


c:\Users\bemne\anaconda3\envs\al\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 620/620 [2:34:45<00:00, 14.98s/it]  


Average loss: 0.21463393649206527


Epoch 2: 100%|██████████| 620/620 [2:23:14<00:00, 13.86s/it]  


Average loss: 0.15654861508387952


Epoch 3: 100%|██████████| 620/620 [2:31:00<00:00, 14.61s/it]  

Average loss: 0.13414811558510747


In [2]:
from tokenizers import BertWordPieceTokenizer